In [1]:
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error
from typing import List

In [2]:
path = os.path.abspath(os.path.join(os.getcwd(), os.path.pardir))

In [3]:
def create_connection(db_file: str) -> sqlite3.Connection:
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [4]:
conn = create_connection(os.path.join(path, "data/league.db"))

In [19]:
sql = """
SELECT 
Club_Name,
COUNT(Club_Name) Total_Games,
AVG(Gold),
SUM(First_Blood) Total_First_Blood,
AVG(Kills)
FROM team_stats
GROUP BY Club_Name
"""
df = pd.read_sql_query(sql, conn)

In [20]:
def covariance(xs: List[float], ys: List[float]) -> float:
    assert len(xs) == len(ys), 'xs and ys must have the same length'
    
    return np.dot(np.mean(xs), np.mean(ys)) / (len(xs) - 1)

In [21]:
covariance(df['Total_Games'], df['AVG(Gold)'])

5.595189206323782

In [22]:
def correlation(xs: List[float], ys: List[float]) -> float:
    std_x = np.std(xs)
    std_y = np.std(ys)
    if std_x > 0 and std_y > 0:
        return covariance(xs, ys) / std_x / std_y
    else:
        return 0

In [23]:
correlation(df['Total_Games'], df['AVG(Gold)'])

0.011731842497421936

In [24]:
df_25_games_plus = df.where(df.Total_Games > 25).dropna()

In [25]:
covariance(df_25_games_plus['Total_Games'], df_25_games_plus['AVG(Gold)'])

13.859411215389308

In [26]:
correlation(df_25_games_plus['Total_Games'], df_25_games_plus['AVG(Gold)'])

0.037210498273098865

In [28]:
covariance(df_25_games_plus['Total_Games'], df_25_games_plus['AVG(Kills)'])

3.0694120122586024

In [29]:
correlation(df_25_games_plus['Total_Games'], df_25_games_plus['AVG(Kills)'])

0.01470471645744044

In [31]:
covariance(df_25_games_plus['Total_Games'], df_25_games_plus['Total_First_Blood'])

13.083979101651531

In [32]:
correlation(df_25_games_plus['Total_Games'], df_25_games_plus['Total_First_Blood'])

0.0025945249803924855

#Interesting fact, there is no correlation between experience in game(count of games) and some basic statistics like average gold or average kills in the game.